In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import matplotlib.dates as mdates
from scipy import stats
warnings.filterwarnings("ignore")


def read_wl_csv(file_path):
    wl_df = pd.read_csv(file_path)

    ##### if the csv is from lighthouse then this drop function is always true
    ##### if the csv is not from lighthouse then you will need to modify the function

    wl_df.drop(labels=range(len(wl_df)-6,len(wl_df)), axis=0, inplace=True)

    keys = wl_df.keys().to_list()
    
    wl_df['date'] = pd.to_datetime(wl_df[keys[0]])
    wl_df[keys[1]].replace([-999, -99, 99, 'NA', 'RM'], np.nan, inplace=True)
    wl_df[keys[2]].replace([-999, -99, 99, 'NA', 'RM'], np.nan, inplace=True)
    wl_df[keys[3]].replace([-999, -99, 99, 'NA', 'RM'], np.nan, inplace=True)
    wl_df['pwl'] = pd.to_numeric(wl_df[keys[1]],errors= 'coerce')
    wl_df['bwl'] = pd.to_numeric(wl_df[keys[2]],errors= 'coerce')
    wl_df['harmwl'] = pd.to_numeric(wl_df[keys[3]],errors= 'coerce')
    wl_df['pwl surge'] = wl_df['pwl'] - wl_df['harmwl']
    wl_df['bwl surge'] = wl_df['bwl'] - wl_df['harmwl']
    wl_df = wl_df.drop(columns=keys[0],axis=0)
    wl_df = wl_df.drop(columns=keys[1],axis=0)
    wl_df = wl_df.drop(columns=keys[2],axis=0)
    wl_df = wl_df.drop(columns=keys[3],axis=0)
    del keys
    return wl_df

def locate_gaps(WL_data):
    lengthMissVal = []
    dates = []
    count = 0
    for i in range(len(WL_data)):
        if pd.isna(WL_data['pwl'][i]):
            if count == 0:  # Start of a new NaN gap
                dates.append(WL_data['date'][i])  # Record the start date of the gap
            count += 1  # Increment the gap length

        else:
            if count > 0:  # End of a NaN gap
                lengthMissVal.append(count)
                count = 0  # Reset count after recording the gap length
    if count > 0:
        lengthMissVal.append(count)

    # Finalize the DataFrame
    WL_data_gaps = pd.DataFrame()
    WL_data_gaps['date'] = pd.to_datetime(dates)
    WL_data_gaps['gapLength'] = lengthMissVal
    WL_data_gaps['gapTime(min)'] = WL_data_gaps['gapLength'] * 6

    del lengthMissVal,dates,count

    return WL_data_gaps

def eligible_gap_length(WL_gaps): #Function to sort the lengh of the gaps into three categories
    WL_gaps_filter_6min = WL_gaps['gapLength'] == 1
    WL_gaps_filter = (WL_gaps['gapLength'] <= 576) & (WL_gaps['gapLength'] > 1)

    #filters the data into individual dataframes
    linear_gaps = WL_gaps[WL_gaps_filter_6min]
    gaps_less_5_days = WL_gaps[WL_gaps_filter]

    del WL_gaps_filter,WL_gaps_filter_6min

    return linear_gaps,gaps_less_5_days


def linear_fill(Wl_data,linear_gaps): #function to fill in gaps with length of 1 using linear approach

    if len(linear_gaps) > 0:

        matching_dates = Wl_data[Wl_data['date'].isin(linear_gaps['date'])]

        index_locations = matching_dates.index.tolist()

        for i in range(len(index_locations)):
            new_value = ((Wl_data.loc[(index_locations[i])-1,'pwl surge']+ Wl_data.loc[index_locations[i]+1,'pwl surge']) / 2) + Wl_data.loc[index_locations[i],'harmwl']
            Wl_data.loc[index_locations[i],'pwl'] = new_value

        del matching_dates, index_locations, new_value
        
        return Wl_data
    
    else:
        print('No single gaps to fill')

        return Wl_data


def check_bwl(Wl_data,gaps):

    if len(gaps) > 0:

        matching_dates = Wl_data[Wl_data['date'].isin(gaps['date'])]

        index_locations = matching_dates.index.tolist()

        gap_length = gaps['gapLength'].tolist()

        valid_gaps = []

        for i in range(len(index_locations)):

            is_valid = Wl_data['bwl'][index_locations[i]:index_locations[i]+gap_length[i]].isna().sum() == 0
            valid_gaps.append(is_valid)
        
        filtered_gaps = gaps[valid_gaps].reset_index(drop=True)

        del matching_dates, index_locations, gap_length, valid_gaps, is_valid

        print(len(filtered_gaps))

        return filtered_gaps
    
    else:
        print('No gaps avaliable to fill')

        return gaps


def poly_gap_fill(Wl_data, gaps):

    if len(gaps) > 0:

        poly_df_list = list()
        
        matching_dates = Wl_data[Wl_data['date'].isin(gaps['date'])]

        index_locations = matching_dates.index.tolist()

        gap_length = gaps['gapLength'].tolist()

        gap_date_list = list()

        adjustment_list = []

        for i in range(len(matching_dates)):

            gap_date_df = pd.DataFrame()

            gap_date_df['date'] = Wl_data['date'][index_locations[i]:index_locations[i]+gap_length[i]]

            gap_date_list.append(gap_date_df)
            


        for i in range(len(index_locations)):

            if index_locations[i]- 2161  > 0 and index_locations[i]+2161+gap_length[i] < len(Wl_data):

                pwl_30_days = Wl_data['pwl'][(index_locations[i]- 2160):index_locations[i]+2160+gap_length[i]].tolist()

                bwl_30_days = Wl_data['bwl'][(index_locations[i]- 2160):index_locations[i]+2160+gap_length[i]].tolist()

                dates = Wl_data['date'][(index_locations[i]- 2160):index_locations[i]+2160+gap_length[i]].tolist()

                linear_df = pd.DataFrame()

                linear_df['pwl 30'] = pwl_30_days
                linear_df['bwl 30'] = bwl_30_days
                linear_df['dates'] = dates

                linear_df.dropna(inplace=True)


                slope, intercept, *_ = stats.linregress(linear_df['bwl 30'],linear_df['pwl 30'])

                poly_df = pd.DataFrame({'bwl': bwl_30_days, 'pwl': pwl_30_days,'date' : pd.to_datetime(dates)})

                poly_df['mwl linear'] = intercept + slope*poly_df['bwl']



                #outliers = poly_df[abs(poly_df['mwl linear'] - poly_df['pwl']) > 0.1]

                #print(f"Number of outliers detected: {len(outliers)}")

                mask = abs(poly_df['mwl linear'] - poly_df['pwl']) > 0.1
                
                poly_df.loc[mask, 'pwl'] = np.nan
                poly_df.loc[mask, 'mwl linear'] = np.nan
                poly_df.loc[mask, 'bwl'] = np.nan
                
                '''plt.scatter(poly_df['pwl surge'], poly_df['bwl surge'])

                plt.scatter(poly_df['mwl surge linear'],poly_df['bwl surge'] , color = 'red', linestyle = 'dashed', label = 'linear model')

                plt.title('pwl surge vs bwl surge')

                plt.show()

                plt.clf()'''
                
            

                
                if poly_df['bwl'].isna().sum() + poly_df['pwl'].isna().sum() < len(Wl_data)*0.1:

                    poly_df_copy = poly_df.copy()

                    poly_df_copy.dropna(inplace=True)

                    poly =np.polynomial.polynomial.Polynomial.fit(poly_df_copy['pwl'],poly_df_copy['bwl'],4)

                    coeffs = np.polyfit(poly_df_copy['bwl'],poly_df_copy['pwl'],4)

                    poly1 = np.poly1d(coeffs)

                    pred_values = poly1(poly_df['bwl'])

                    poly_df['mwl'] = pred_values

                    '''plt.scatter([poly_df['pwl surge']], poly_df['bwl surge'])

                    plt.scatter(poly_df['mwl surge'],poly_df['bwl surge'], color = 'black', linestyle = 'dashed')

                    plt.title('pwl surge vs bwl surge (poly)')

                    plt.show()

                    plt.clf()'''

                    poly_df_list.append(poly_df)

                    del poly_df_copy, poly, pred_values


                else:
                   print('Can not fill gap not enough points')


            else:
                print('Can not fill gap out of bounds')
        
        print(len(index_locations))
        return poly_df_list, index_locations, gap_length, gap_date_list, adjustment_list


    else:
        print('No gaps to Fill')
        index_locations  = []
        gap_length = []
        gap_date_list=[]

        poly_df_list = []

        return poly_df_list, index_locations, gap_length, gap_date_list

def fill_gaps(poly_list, gap_dates_list, wl_df):
    matched_dates1 = []
    matched_dates2 = []

    # Process each gap and its corresponding poly_df
    for gap_df, poly_df in zip(gap_dates_list, poly_list):
        # Ensure dates are in datetime format
        gap_df['date'] = pd.to_datetime(gap_df['date'])
        poly_df['date'] = pd.to_datetime(poly_df['date'])

        # Find common dates between gap_df and poly_df
        common_dates = gap_df['date'][gap_df['date'].isin(poly_df['date'])]

        # Filter both DataFrames for the common dates
        filtered_gap_df = gap_df[gap_df['date'].isin(common_dates)]
        filtered_poly_df = poly_df[poly_df['date'].isin(common_dates)]

        # Append the filtered DataFrames
        matched_dates1.append(filtered_gap_df)
        matched_dates2.append(filtered_poly_df)

    # Combine all matched DataFrames
    match_df_1 = pd.concat(matched_dates1, ignore_index=True)
    match_df_2 = pd.concat(matched_dates2, ignore_index=True)

    # Merge the poly_df data with the original wl_df
    Wl_data_total = match_df_2.merge(wl_df, on='date', how='outer')

    # Clean up columns
    Wl_data_total = Wl_data_total.drop(columns=['bwl_x', 'pwl_x', '#date+time'], errors='ignore')
    Wl_data_total['pwl'] = Wl_data_total.get('pwl_y', None)
    Wl_data_total['bwl'] = Wl_data_total.get('bwl_y', None)
    Wl_data_total = Wl_data_total.drop(columns=['pwl_y', 'bwl_y'], errors='ignore')

    # Clean up memory by deleting unnecessary variables
    del poly_list, gap_dates_list, matched_dates1, matched_dates2, match_df_1, match_df_2

    return Wl_data_total  


def adjustment(filled_df, filtered_gaps, index_locations):



    average_before_gap = np.nanmean(filled_df['pwl'][(index_locations[i]-6):index_locations[i]].tolist())
    average_after_gap = np.nanmean(filled_df['pwl'][(index_locations[i]+1+gap_length[i]):index_locations[i]+6+gap_length[i]].tolist())

    n_length = gap_length[i]

    for k in range(n_length):

        adjustment_value = (average_after_gap + (k / n_length)) * (average_before_gap - average_after_gap)

        adjustment_list.append(adjustment_value)

    adj_df = pd.DataFrame()
    adj_df['values'] = adj_list

    mwl_adj_df = filled_df.dropna(subset='mwl')

    mwl_adj_df['mwl adjusted'] = mwl_adj_df['mwl']

    mwl_adj_df['mwl adjusted'] = mwl_adj_df['mwl adjusted'] + adj_df['values']

    print(mwl_adj_df)


    return filled_df
        

def create_gaps(dataset):

    import random

    wl_data =  dataset.copy() #pd.DataFrame(dataset)

    random_index = [random.randint(0,len(wl_data))for _ in range(1000)]

    max_gap_size = 100
    random_index = random.sample(range(len(wl_data) - max_gap_size), 1000)


    #create one six min gap

    wl_data.loc[random_index[0], 'pwl'] = np.nan
    random_index = random_index[1:]


    # create 5 30 min gaps

    for i in range(5):

        wl_data.loc[random_index[i]:random_index[i] + 4, 'pwl'] = np.nan
    
    random_index = random_index[5:]

    #create 10 1hr gaps

    for i in range(10):

        wl_data.loc[random_index[i]:random_index[i] + 9, 'pwl'] = np.nan
    
    random_index = random_index[10:]

    #creates 50 5 hr gaps

    for i in range(50):

        wl_data.loc[random_index[i]:random_index[i] + 49, 'pwl'] = np.nan
    
    random_index = random_index[50:]

    #creates 100 10hr gaps

    for i in range(100):

        wl_data.loc[random_index[i]:random_index[i] + 99, 'pwl'] = np.nan
    
    random_index = random_index[100:]


    #print((wl_data.isna().sum()))

    return wl_data

def cbi_gapfill(filepath):

    print('Reading dataset')
    wl_dataset = read_wl_csv(filepath)

    gaps_true = input('Do you want to create artifical gaps y/n? ')

    if str(gaps_true) == str('y'):
        
        wl_dataset_gaps = create_gaps(wl_dataset)

        print('Gaps Created')

        Wl_gaps = locate_gaps(wl_dataset_gaps)

        print('Total number of gaps: ', len(Wl_gaps))

        linear_gaps,multi_gaps = eligible_gap_length(Wl_gaps)

        print('Number of Linear Gaps filled:', len(linear_gaps))

        dataset_LF = linear_fill(wl_dataset_gaps,linear_gaps)

        print('Single gaps filled')

        valid_multi_gaps = check_bwl(dataset_LF,multi_gaps)

        print('Number of gaps with backup water level:', len(valid_multi_gaps))

        poly_wl_list, index_location, gap_length, gap_list, adjustment_list = poly_gap_fill(dataset_LF,valid_multi_gaps)


        if len(poly_wl_list) > 0 :

            filled_df = fill_gaps(poly_wl_list,gap_list,dataset_LF)

            adj_values = adjustment(filled_df, adjustment_list)

            print('Gaps filled', + len(poly_wl_list))

            return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, adj_values
        
        else:
            adj_values = []

            return dataset_LF, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, adj_values

    elif str(gaps_true) == str('n'):

        Wl_gaps = locate_gaps(wl_dataset)

        print('Total number of gaps: ', len(Wl_gaps))

        linear_gaps,multi_gaps = eligible_gap_length(Wl_gaps)

        print('Number of Linear Gaps filled:', len(linear_gaps))

        dataset_LF = linear_fill(wl_dataset,linear_gaps)

        print('Single gaps filled')

        valid_multi_gaps = check_bwl(dataset_LF,multi_gaps)

        print('Number of gaps with backup water level:', len(valid_multi_gaps))

        poly_wl_list, index_location, gap_length, gap_list, adjustment_list = poly_gap_fill(dataset_LF,valid_multi_gaps)


        if len(poly_wl_list) > 0 :

            filled_df = fill_gaps(poly_wl_list,gap_list,dataset_LF)

            adj_values = adjustment(filled_df, adjustment_list)

            print('Gaps filled', + len(poly_wl_list))

            return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, adj_values
        
        else:
            adj_values = []

            return dataset_LF, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, adj_values
    else:
        print('Not an acceptable answer')
        dataset_LF = []
        Wl_gaps = []
        dataset_LF = []
        poly_wl_list = []
        gap_list = []
        adj_values = []
        
        return filled_df, wl_dataset, Wl_gaps, dataset_LF, poly_wl_list, gap_list, adj_values



    

filled_df, wl_dataset, all_gaps, dataset_LF, poly_wl, filled_gap_list, adj_values = cbi_gapfill(r'C:\Users\mrpro\Documents\Code\CBI\Gap_Filling\P21_2016_gaps.csv')


Reading dataset
Gaps Created
Total number of gaps:  169
Number of Linear Gaps filled: 5
Single gaps filled
139
Number of gaps with backup water level: 139
Can not fill gap out of bounds
Can not fill gap out of bounds
Can not fill gap out of bounds
Can not fill gap out of bounds
Can not fill gap out of bounds
Can not fill gap out of bounds
Can not fill gap out of bounds
139


TypeError: adjustment() missing 1 required positional argument: 'index_locations'

In [ ]:
filled_df['pwl actual'] = wl_dataset['pwl']
filled_df

,date,mwl linear,mwl,harmwl,pwl surge,bwl surge,pwl,bwl,pwl actual
0,2016-01-01 00:00:00,NaN,NaN,1.436,0.369,1.183,1.805,2.619,1.805
1,2016-01-01 00:06:00,NaN,NaN,1.441,0.367,1.181,1.808,2.622,1.808
2,2016-01-01 00:12:00,NaN,NaN,1.446,0.380,1.188,1.826,2.634,1.826
3,2016-01-01 00:18:00,NaN,NaN,1.451,0.392,1.196,1.843,2.647,1.843
4,2016-01-01 00:24:00,NaN,NaN,1.456,0.394,1.194,1.850,2.650,1.850
...,...,...,...,...,...,...,...,...,...
87835,2016-12-31 23:30:00,NaN,NaN,1.536,0.475,1.220,2.011,2.756,2.011
87836,2016-12-31 23:36:00,NaN,NaN,1.541,0.483,1.225,2.024,2.766,2.024
87837,2016-12-31 23:42:00,NaN,NaN,1.546,0.476,1.219,2.022,2.765,2.022
87838,2016-12-31 23:48:00,NaN,NaN,1.551,0.478,1.219,2.029,2.770,2.029


In [ ]:
filled_df['error'] = abs((filled_df['mwl'] - filled_df['pwl actual']) / filled_df['pwl actual']) *100
filled_df['adjusted error'] = abs((filled_df['mwl adjusted'] - filled_df['pwl actual']) / filled_df['pwl actual']) *100
filled_df

KeyError: 'mwl adjusted'

In [ ]:
filled_df[3590:3602]

,date,mwl linear,mwl,harmwl,pwl surge,bwl surge,pwl,bwl,mwl adjusted,pwl actual,error,adjusted error
3590,2016-01-15 23:00:00,NaN,NaN,1.386,0.423,1.242,1.809,2.628,NaN,1.809,NaN,NaN
3591,2016-01-15 23:06:00,NaN,NaN,1.391,0.406,1.228,1.797,2.619,NaN,1.797,NaN,NaN
3592,2016-01-15 23:12:00,NaN,NaN,1.395,0.416,1.236,1.811,2.631,NaN,1.811,NaN,NaN
3593,2016-01-15 23:18:00,NaN,NaN,1.400,0.442,1.256,1.842,2.656,NaN,1.842,NaN,NaN
3594,2016-01-15 23:24:00,NaN,NaN,1.405,0.433,1.249,1.838,2.654,NaN,1.838,NaN,NaN
3595,2016-01-15 23:30:00,NaN,NaN,1.410,0.409,1.229,1.819,2.639,NaN,1.819,NaN,NaN
3596,2016-01-15 23:36:00,NaN,NaN,1.415,0.401,1.222,1.816,2.637,NaN,1.816,NaN,NaN
3597,2016-01-15 23:42:00,NaN,NaN,1.420,0.401,1.223,1.821,2.643,NaN,1.821,NaN,NaN
3598,2016-01-15 23:48:00,NaN,NaN,1.425,0.402,1.222,1.827,2.647,NaN,1.827,NaN,NaN
3599,2016-01-15 23:54:00,NaN,NaN,1.430,0.392,1.215,1.822,2.645,NaN,1.822,NaN,NaN


In [ ]:
filled_df.describe()

,date,mwl linear,mwl,harmwl,pwl surge,bwl surge,pwl,bwl,mwl adjusted,pwl actual,error,adjusted error
count,87840,5809.000000,5809.000000,87840.000000,86132.000000,86275.000000,74489.000000,86275.000000,5580.000000,86132.000000,5702.000000,5473.000000
mean,2016-07-01 23:57:00,1.838498,1.838144,1.555959,0.263662,1.062536,1.817205,2.617894,1.755798,1.818899,0.285444,21.539753
min,2016-01-01 00:00:00,0.890919,0.892175,1.035000,-0.617000,-1.609000,0.585000,0.000000,0.547577,0.585000,0.000024,0.001188
25%,2016-04-01 11:58:30,1.697219,1.696844,1.458000,0.187000,0.994000,1.693000,2.526000,1.335584,1.692000,0.108294,8.376366
50%,2016-07-01 23:57:00,1.854125,1.854986,1.572000,0.262000,1.056000,1.834000,2.628000,1.749761,1.835000,0.231685,20.071090
75%,2016-10-01 11:55:30,1.995413,1.995399,1.675000,0.342000,1.128000,1.962000,2.722000,2.112593,1.966000,0.390440,31.703504
max,2016-12-31 23:54:00,2.310423,2.313729,1.917000,0.894000,1.593000,2.507000,3.124000,3.302941,2.507000,2.583990,70.952498
std,NaN,0.220967,0.221213,0.158702,0.147163,0.115146,0.220411,0.158955,0.520910,0.220358,0.240393,15.285675
